In [1]:
import pandas as pd
from unidecode import unidecode

# 데이터 로드
df = pd.read_csv('/Users/AIFFELthon/final/data/modified_data_03.csv')

# 결측값 처리
df.loc[df['id'] == 119609, 'slug'] = 'nan'
df.loc[df['id'] == 119609, 'name'] = 'NaN'
df.loc[df['id'] == 100122, 'slug'] = 'null'
df.loc[df['id'] == 100122, 'name'] = 'NULL'
df.loc[df['id'] == 468408, 'slug'] = 'none'
df.loc[df['id'] == 468408, 'name'] = 'None'

# 필요한 열만 선택
game_data = df[['id', 'name', 'description', 'released', 'rating', 'genre', 'theme', 'developers', 'publishers', 'tags']]

# 열을 문자열로 변환하고 소문자로 변환
columns_to_convert = ['genre', 'theme', 'developers', 'publishers', 'tags']
for column in columns_to_convert:
    game_data[column] = game_data[column].astype(str).str.lower()
    # 다국어 텍스트를 ASCII로 변환 (선택 사항)
    game_data[column] = game_data[column].apply(lambda x: unidecode(x))

# ','를 ' '로 대체
for column in columns_to_convert:
    game_data[column] = game_data[column].str.replace(',', ' ', regex=False)

/var/folders/zw/1zmd_ykn3zbd2m00vrvhv5m40000gn/T/ipykernel_42629/1021161157.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_data[column] = game_data[column].astype(str).str.lower()
/var/folders/zw/1zmd_ykn3zbd2m00vrvhv5m40000gn/T/ipykernel_42629/1021161157.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_data[column] = game_data[column].apply(lambda x: unidecode(x))
/var/folders/zw/1zmd_ykn3zbd2m00vrvhv5m40000gn/T/ipykernel_42629/1021161157.py:21: SettingWithCopyWarning: 
A value is tryin

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# 콘텐츠 기반 필터링을 위해 장르, 테마, 태그를 하나의 문자열로 결합
game_data['combined_features'] = (
    (game_data['genre'] + ' ') * 6 + 
    (game_data['theme'] + ' ') * 4 +  
    game_data['tags']
)

# TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=0.8)
tfidf_matrix = tfidf.fit_transform(game_data['combined_features'])

# NearestNeighbors 모델 사용
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(tfidf_matrix)

# 게임 이름과 인덱스 매핑
indices = pd.Series(game_data.index, index=game_data['name']).drop_duplicates()

# 게임 추천 함수
def get_recommendations(title, nn_model, tfidf_matrix, indices, n_recommendations=10):
    if title not in indices:
        return "Game not found in dataset."
    
    idx = indices[title]
    query_vector = tfidf_matrix[idx]
    distances, indices = nn_model.kneighbors(query_vector, n_neighbors=n_recommendations + 1)
    
    sim_scores = list(zip(indices.flatten(), distances.flatten()))
    sim_scores = sorted(sim_scores, key=lambda x: x[1])
    game_indices = [i[0] for i in sim_scores[1:]]
    
    recommendations = game_data[['name', 'genre', 'theme']].iloc[game_indices]
    recommendations['distance'] = [score[1] for score in sim_scores[1:]]
    
    return recommendations

/var/folders/zw/1zmd_ykn3zbd2m00vrvhv5m40000gn/T/ipykernel_42629/597158268.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_data['combined_features'] = (


In [3]:
df[df['name'] == 'League of Legends'][['name', 'genre', 'theme']]

,name,genre,theme
550345,League of Legends,"Action,MOBA,RPG,Strategy",


In [4]:
get_recommendations('League of Legends', nn, tfidf_matrix, indices, 10)

,name,genre,theme,distance
199795,王者英雄 The Great Hero,action moba rpg strategy,,0.042614
168740,Stellar Stars,action moba rpg strategy,,0.055819
469443,Heroes of the Storm,action moba rpg strategy,adult,0.061366
146123,Arena of Valor,action moba rpg,,0.135038
66352,Fates Forever,action moba rpg,,0.142709
16542,UnnyWorld (itch),action moba rpg,,0.185269
101063,Blade Reborn,action moba rpg,,0.194928
509215,MEDLand Mayhem Windows + macOS,moba rpg strategy,,0.321784
91050,Brawl Swords,moba rpg strategy,,0.323819
60258,Clash of Exiles,action moba rpg rts,,0.332545


In [5]:
def filter_games(genre=None, theme=None, developers=None, publishers=None, tags=None):
    filtered_data = game_data
    
    if genre:
        genre = unidecode(genre.lower())
        filtered_data = filtered_data[filtered_data['genre'].str.contains(genre, case=False)]
    
    if theme:
        theme = unidecode(theme.lower())
        filtered_data = filtered_data[filtered_data['theme'].str.contains(theme, case=False)]
    
    if developers:
        developers = unidecode(developers.lower())
        filtered_data = filtered_data[filtered_data['developers'].str.contains(developers, case=False)]
    
    if publishers:
        publishers = unidecode(publishers.lower())
        filtered_data = filtered_data[filtered_data['publishers'].str.contains(publishers, case=False)]

    if tags:
        tags = unidecode(tags.lower())
        filtered_data = filtered_data[filtered_data['tags'].str.contains(tags, case=False)]
    
    return filtered_data

In [6]:
filtered_games = filter_games(genre='action', developers='riot games')
filtered_games.head()

,id,name,description,released,rating,genre,theme,developers,publishers,tags,combined_features
4533,158539,Derby Rumble,Derby Rumble is a game that will be sure to br...,2017-10-02,0.00,action,,dark chariot games,,2d pixel graphics futuristic fast-paced destru...,action action action action action action ...
192997,383521,Project F: Riot’s Action RPG,,,0.00,action rpg,,riot games,,,action rpg action rpg action rpg action rpg ac...
334629,383516,League of Legends: Wild Rift,League of Legends: Wild Rift lets you dive int...,2020-10-27,3.84,action moba strategy,,riot games,riot games,fantasy moba,action moba strategy action moba strategy acti...
550345,23598,League of Legends,2024-04-23T14:27:06,2009-10-27,3.64,action moba rpg strategy,,riot games,riot games,multiplayer online multiplayer moba,action moba rpg strategy action moba rpg strat...


In [ ]:
df[df['name'] == 'League']